In [1]:
# WORKING VERSION OF COMBINED DRAFTS
# Create 327x3 matrix, for each image set contains neutral image name, expression image name, emotion label
# Generate normalized Euclidean Differences array
# Generate target values array
# Feed to SVM, evaulate with Cross Validation
import numpy as np
import cv2
import dlib
import os
import math
from sklearn import svm
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

# Function that returns the distance between two given points
def ptDist(pt1, pt2):
    x1 = pt1[0]
    x2 = pt2[0]
    y1 = pt1[1]
    y2 = pt2[1]
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

# Function that returns array of 68 landmarks for given image name
def get68LMA(imgName):
    predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    image = cv2.imread(imgName, 1)
    
    # Detect face(s)
    dets = detector(image, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    for d in dets:
        shape = predictor(image, d)
        for index in range(0, 68):
            point = shape.part(index)
            tuplepoint = (point.x, point.y)
            LMA.append(tuplepoint)
    return LMA

# Function that eturns normalized array of Euclidean distances given two arrays of coordinates
def getEucDist(nLA, eLA):
    LDA = []
    sum = 0
    # Get array of distances
    for pt in range(68):
        dif = ptDist(eLA[pt], nLA[pt])     
        LDA.append(dif)
        sum += dif
    avg = sum/68
    # Normalize array
    for i in range(68):
        LDA[i] -= avg      
    return LDA

# Function that returns 13-point numpy vector of distances between selected physiological pts, given the 68-point vector
def getPDA(iLA):
    PDA = []
    PDA.append(ptDist(iLA[21], iLA[22])) #distance 0: distance between brows
    PDA.append(ptDist(iLA[21], iLA[27])) #distance 1: inner brow to nose- left
    PDA.append(ptDist(iLA[22], iLA[27])) #distance 2: inner brow to nose- right
    PDA.append(ptDist(iLA[21], iLA[39])) #distance 3: inner brow to inner eye corner- left
    PDA.append(ptDist(iLA[22], iLA[42])) #distance 4: inner brow to inner eye corner- right
    PDA.append(ptDist(iLA[36], iLA[48])) #distance 5: outer eye corner to outer mouth corner- left
    PDA.append(ptDist(iLA[45], iLA[54])) #distance 6: outer eye corner to outer mouth corner- right
    PDA.append(ptDist(iLA[48], iLA[54])) #distance 7: distance between mouth corners
    PDA.append(ptDist(iLA[62], iLA[66])) #distance 8: height of mouth
    PDA.append(ptDist(iLA[36], iLA[50])) #distance 9: raising of upper lip
    PDA.append(ptDist(iLA[45], iLA[52])) #distance 10: raising of upper lip
    PDA.append(ptDist(iLA[19], iLA[37])) #distance 11: brow arch- left
    PDA.append(ptDist(iLA[24], iLA[44])) #distance 12: brow arch- right
    return np.asarray(PDA)

"""MAIN PROGRAM BEGINS"""
# Get paths to face images and emotion labels
imagedir = []
labeldir = []
root, dirs, files = os.walk("./cohn-kanade-images2",).__next__()
for d in dirs:
    imagedir.append(os.path.join(root,d))
    labeldir.append(os.path.join('./Emotion',d))

# initialize variables
dataset = []
NLA = []
ELA = []

# look over all directories of individuals S001 through S999
for d in dirs:
    for dd in os.listdir(os.path.join('./cohn-kanade-images2',d)):
        # skip over .DS_Store files
        if 'Store' not in dd:
            curdir = os.path.join('./cohn-kanade-images2',d,dd)
            files = os.listdir(curdir)
            baseimage = os.path.join(curdir,files[0])
            faceimage = os.path.join(curdir,files[-1])

            # get directories of multiple expressions of same person
            labeldir = os.path.join('./Emotion',d,dd)
            if os.path.isdir(labeldir):
                labels = os.listdir(labeldir)
                # read label file if it exists
                if len(labels)>0:
                    f=open(os.path.join(labeldir,labels[0]),"r")
                    for line in f:
                        targetlabel = float(line)
                    f.close()
                    
                    # append label to dataset
                    dataset.append([baseimage, faceimage, targetlabel])

"""68-POINT LANDMARK DETECTION MODEL"""
# Get array of Euclidean differences between neutral and expression 68-landmark arrays
tempLDA = []
for imgIndex in range(len(dataset)):
    nImgName = dataset[imgIndex][0]
    eImgName = dataset[imgIndex][1]
    nLA = get68LMA(nImgName)
    eLA = get68LMA(eImgName)
    NLA.append(nLA)
    ELA.append(eLA)
    tempLDA.append(getEucDist(nLA, eLA))
LDA = np.asmatrix(tempLDA)

# Create target value array for expression images
tempTargetValues = []
for imgSet in range(len(dataset)):
    tempTargetValues.append(dataset[imgSet][2])
targetValues = np.asarray(tempTargetValues)

# # Feed LDA and targetValues into SVM, use cross validation, generate confusion matrix
# x_train, x_test, y_train, y_test = cross_validation.train_test_split(LDA, targetValues, test_size=0.4, random_state=2)
# normCLF = svm.SVC(kernel='linear').fit(x_train, y_train)
# predictions = normCLF.predict(x_test)
# cmatrix = confusion_matrix(y_test, predictions)

# # Format CMatrix to show percentages, rounded to 1 decimal point
# np.set_printoptions(precision=1)
# rowSums = np.sum(cmatrix, axis=1)
# pCMatrix = np.divide(cmatrix*100., rowSums, dtype=float)
# print pCMatrix


"""PHYSIOLOGICALLY BASED LANDMARK DETECTION MODEL"""
# Get array of corresponding differences between 13 distances of base images and expression images
tempPDA = []
for i in range(len(NLA)):
    tempPDA.append(getPDA(NLA[i])-getPDA(ELA[i]))
PDA = np.asmatrix(tempPDA)

# Feed PDA and targetValues into SVM, use cross validation, generate confusion matrix
x_train2, x_test2, y_train2, y_test2 = cross_validation.train_test_split(PDA, targetValues, test_size=0.4, random_state=3)
linCLF = svm.SVC(kernel='linear',probability=True).fit(x_train2, y_train2)
predictions2 = linCLF.predict(x_test2)
cMatrix2 = confusion_matrix(y_test2, predictions2)

# Format CMatrix2 to show percentages, already rounded to 1 decimal point
# rowSums2 = np.sum(cMatrix2, axis=1)
# pCMatrix2 = np.divide(cMatrix2*100., rowSums2, dtype=float)
print(cMatrix2)

joblib.dump(linCLF, 'linear7.pkl') 

[[11  2  2  0  0  0  0]
 [ 1  6  0  0  0  0  0]
 [ 1  1 18  0  0  0  0]
 [ 1  0  0  7  1  2  0]
 [ 0  0  1  0 27  0  0]
 [ 5  0  0  3  0  6  1]
 [ 0  1  0  1  0  1 32]]


['linear7.pkl',
 'linear7.pkl_01.npy',
 'linear7.pkl_02.npy',
 'linear7.pkl_03.npy',
 'linear7.pkl_04.npy',
 'linear7.pkl_05.npy',
 'linear7.pkl_06.npy',
 'linear7.pkl_07.npy',
 'linear7.pkl_08.npy',
 'linear7.pkl_09.npy',
 'linear7.pkl_10.npy',
 'linear7.pkl_11.npy']

In [14]:
import pickle

with open("/Users/kaili/Dropbox/HSSF1617/NeutralLMLocs.pkl", 'wb') as f:
    pickle.dump(NLA, f)
f.close()
with open("/Users/kaili/Dropbox/HSSF1617/ExpLMLocs.pkl", 'wb') as f:
    pickle.dump(ELA, f)
f.close()

In [41]:
linCLF.coef_.shape

(21, 13)

In [101]:
def getPDA(iLA):
    PDA = []
    PDA.append(ptDist(iLA[21], iLA[22])) #distance 0: distance between brows
    PDA.append(ptDist(iLA[21], iLA[27])) #distance 1: inner brow to nose- left
    PDA.append(ptDist(iLA[22], iLA[27])) #distance 2: inner brow to nose- right
    PDA.append(ptDist(iLA[21], iLA[39])) #distance 3: inner brow to inner eye corner- left
    PDA.append(ptDist(iLA[22], iLA[42])) #distance 4: inner brow to inner eye corner- right
    PDA.append(ptDist(iLA[36], iLA[48])) #distance 5: outer eye corner to outer mouth corner- left
    PDA.append(ptDist(iLA[45], iLA[54])) #distance 6: outer eye corner to outer mouth corner- right
    PDA.append(ptDist(iLA[48], iLA[54])) #distance 7: distance between mouth corners
    PDA.append(ptDist(iLA[62], iLA[66])) #distance 8: height of mouth
    PDA.append(ptDist(iLA[36], iLA[50])) #distance 9: raising of upper lip
    PDA.append(ptDist(iLA[45], iLA[52])) #distance 10: raising of upper lip
    PDA.append(ptDist(iLA[19], iLA[37])) #distance 11: brow arch- left
    PDA.append(ptDist(iLA[24], iLA[44])) #distance 12: brow arch- right
    return np.asarray(PDA)

tempPDA = []
for i in range(len(NLA)):
    tempPDA.append(getPDA(NLA[i])-getPDA(ELA[i]))
PDA = np.asmatrix(tempPDA)

linCLF = svm.LinearSVC().fit(x_train2, y_train2)
predictions2 = linCLF.predict(x_test2)
cMatrix2 = confusion_matrix(y_test2, predictions2)
print(cMatrix2)

joblib.dump(linCLF, 'workingLinearSVM.pkl') 

[[11  4  0  0  0  0  0]
 [ 1  6  0  0  0  0  0]
 [ 4  0 16  0  0  0  0]
 [ 1  0  0  6  1  2  1]
 [ 0  1  0  0 27  0  0]
 [ 6  0  1  2  0  6  0]
 [ 0  0  0  0  0  0 35]]


['workingLinearSVM.pkl',
 'workingLinearSVM.pkl_01.npy',
 'workingLinearSVM.pkl_02.npy',
 'workingLinearSVM.pkl_03.npy']

In [208]:
from sklearn.externals import joblib
from sklearn import svm


loaded = joblib.load('theModel.pkl')
predictionsasdfsdf = loaded.predict(x_test2)
print(confusion_matrix(y_test2, predictionsasdfsdf))

[[11  2  1  0  0  0  1]
 [ 0  7  0  0  0  0  0]
 [ 1  0 19  0  0  0  0]
 [ 1  1  0  7  1  1  0]
 [ 0  0  1  0 27  0  0]
 [ 1  5  0  2  1  6  0]
 [ 0  2  0  2  0  0 31]]


In [62]:
joblib.dump(linCLF, 'workingLinearSVM.pkl') 

['workingLinearSVM.pkl',
 'workingLinearSVM.pkl_01.npy',
 'workingLinearSVM.pkl_02.npy',
 'workingLinearSVM.pkl_03.npy']

In [102]:
PDA[13]

matrix([[-13.03770501,  -2.54412371,  -3.23733987,   1.60101977,
           2.58383776,  20.25504484,  21.54680737, -37.02862209,
         -16.        ,   7.42182001,  12.26044002,  -4.04097145,
          -2.95473831]])

In [187]:
dataset[11][1]

'./cohn-kanade-images2/S014/002/S014_002_00000016.png'

In [103]:
linCLF.predict(np.asarray([2,-1,4,-2,5,2,19,14,-19,-11,9,-3,-2]).reshape(1,-1))

array([ 5.])

In [188]:
PDA[11]

matrix([[ 2.994,  3.399,  1.664,  0.267, -0.539, -2.595, -0.717,  2.995,
         -1.   ,  4.965,  6.064,  2.046,  2.043]])

In [185]:
#emo = [2, -1,    4, -2,   5,  2,  19, 14, -19, -11, 9, -3, -2]
#emo = [0, -27, -23, -29, -28, 28, 12, -5, 0, -15, -11, -4, -6]
emo =  [0, -2,    5, 0,   0,  0,  12, 0, -19, -11, 0, -4, -6]
linCLF.predict(np.asarray(emo).reshape(1,-1))

array([ 5.])

In [194]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

print(linCLF.coef_[5])

[ 0.349 -0.331  0.37  -0.118 -0.071 -0.238 -0.011  0.046  0.069  0.015
  0.035  0.061 -0.008]


In [202]:
#emo = [3, 3,   2,   0,   -1, -3,  -1,  3, -1, 5,    6,  2,  2]
#emo = [5, -32, -28, -34, -33, -15, -31, 3, 1, -14, -12, -2, -3]
emo = [5, -32, -28, -34, -33, -15, -31, 0, 0, 0, 0, 0, 0]
linCLF.predict(np.asarray(emo).reshape(1,-1)/5)


## sadness = first three displacements most important
#### make a transformation matrix to scale first 7 displacements by 1/5 and set last 6 to 0 since they never change

array([ 6.])

In [206]:
#####


# SUCCESSFUL SADNESS TRANSFORMATION


#####

TRANS_MAT = np.asmatrix([[.2,0,0,0,0,0,0,0,0,0,0,0,0], 
             [0,.2,0,0,0,0,0,0,0,0,0,0,0], 
             [0,0,.2,0,0,0,0,0,0,0,0,0,0], 
             [0,0,0,0,.2,0,0,0,0,0,0,0,0], 
             [0,0,0,0,0,.2,0,0,0,0,0,0,0], 
             [0,0,0,0,0,0,.2,0,0,0,0,0,0], 
             [0,0,0,0,0,0,0,.2,0,0,0,0,0], 
             [0,0,0,0,0,0,0,0,0,0,0,0,0], 
             [0,0,0,0,0,0,0,0,0,0,0,0,0], 
             [0,0,0,0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,0,0,0], 
             [0,0,0,0,0,0,0,0,0,0,0,0,0], 
             [0,0,0,0,0,0,0,0,0,0,0,0,0]])
ARGO_EXP = np.asmatrix([5, -32, -28, -34, -33, -15, -31, 3, 1, -14, -12, -2, -3])
linCLF.predict(np.asarray(ARGO_EXP*TRANS_MAT).reshape(1,-1))

array([ 6.])

In [207]:
ARGO_EXP2 = [-11, -12, -9, -5, -7, 28, 25, -3, 2, 5, 9, 2, 5]
linCLF.predict(np.asarray(ARGO_EXP2*TRANS_MAT).reshape(1,-1))

array([ 2.])